# Running PyOPIA on EDITO Datalab
This notebook demontrates and end-to-end pipeline analysis of a small image dataset using PyOPIA running on EDITO Datalab.

Read more about PyOPIA [here](https://pyopia.readthedocs.io/en/latest/intro.html).

In [ ]:
packages = [
    "pyopia[classification]",
    "ipywidgets",
    "directory_tree",
    "matplotlib",
]

In [ ]:
for package in packages:
    !uv pip install --prefix=".local" {package}

In [ ]:
# Fix a dask issue - maybe related to EDITO container system version of packages
!uv pip install --prefix=".local" --force-reinstall --upgrade xarray==2023.12.0 zarr dask fsspec

In [ ]:
#
# We need to manually add the package location used for uv pip install above,
# since we cannot install into the system python site-package directory (read-only)
#
import sys

sys.path.insert(0, ".local/lib/python3.12/site-packages")

In [ ]:
import contextlib
import tempfile
from collections import namedtuple
from pathlib import Path
import zipfile
import dask.diagnostics
import directory_tree
import matplotlib.pyplot as plt
import numpy as np
import pyopia
import pyopia.cli
import pyopia.instrument.silcam
import pyopia.io
import pyopia.statistics
import pyopia.dataexport.ecotaxa

In [ ]:
from xarray.core.parallelcompat import list_chunkmanagers

list_chunkmanagers()

In [ ]:
# PyOPIA version
pyopia.__version__

In [ ]:
# We don't care about warnings
import warnings
import os
warnings.filterwarnings('ignore')

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [ ]:
def calculcate_copepod_concentration(
    xstats,
    image_stats,
    cop_prob_lim=0.7,
    path_length_mm=30,
):
    """Calculate copepod concentration (#/L) from PyOPIA stats"""

    # Get xstats for probable copepod particles
    xstats_cop = xstats.where(xstats["probability_copepod"] > cop_prob_lim).dropna(
        dim="index"
    )

    # Recreate PyOPIA config from xstats
    conf = pyopia.io.steps_from_xstats(xstats)
    pixel_size = conf["general"]["pixel_size"]

    # Get number of images
    num_images = image_stats.timestamp.size

    # Get raw image shape
    imx, imy, _ = xstats_cop.attrs["raw_image_shape"]

    # Calculate sample volume
    sample_volume = pyopia.statistics.get_sample_volume(
        pixel_size, path_length_mm, imx=imx, imy=imy
    )

    # Calculate total number of copepods per liter based on total volume sampled
    total_sample_volume = float(sample_volume * num_images)
    num_copepods = xstats_cop.index.size
    copepods_per_litre = float(num_copepods / total_sample_volume)

    return_tuple = namedtuple(
        "CopepodEstimates",
        ["copepods_per_litre", "num_copepods", "total_sample_volume", "num_images"],
    )

    return return_tuple(
        copepods_per_litre=copepods_per_litre,
        num_copepods=num_copepods,
        total_sample_volume=total_sample_volume,
        num_images=num_images,
    )

In [ ]:
#
# Create a temporary directory for PyOPIA project data
#
proj_dir_tmp = tempfile.TemporaryDirectory()
proj_dir = Path(proj_dir_tmp.name) / Path("pyopiaproject")
proj_name = proj_dir.name
proj_dir

In [ ]:
#
# Initialize PyOPIA project with example data
#
with contextlib.chdir(proj_dir.parent):
    pyopia.cli.init_project(proj_name, instrument="silcam", example_data=True)

In [ ]:
#
# Show the newly created PyOPIA project directory structure
#
directory_tree.DisplayTree(proj_dir, header=True)

In [ ]:
#
# Load PyOPIA config, get stats file and processed directory paths
#
conf = pyopia.io.load_toml(proj_dir / Path("config.toml"))

stats_file = proj_dir / Path(conf["steps"]["output"]["output_datafile"] + "-STATS.nc")
processed_dir = stats_file.parent

conf

In [ ]:
#
# Generate list of images to be processed
#
with contextlib.chdir(proj_dir):
    img_files = sorted(Path("images").glob("*.silc"))
img_files

In [ ]:
#
# Process images
#
with contextlib.chdir(proj_dir):
    # Initialise the pipeline and run the initial steps
    processing_pipeline = pyopia.pipeline.Pipeline(conf)

    for filename in img_files:
        # Process the image to obtain the stats dataframe
        stats = processing_pipeline.run(str(filename))

In [ ]:
#
# Merge per-image stats files into one netcdf
#
with contextlib.chdir(proj_dir):
    pyopia.io.merge_and_save_mfdataset(processed_dir, prefix="pyopiaproject-")

In [ ]:
#
# Load stats
#
xstats = pyopia.io.load_stats(str(stats_file))
stats = xstats.to_pandas()
stats["depth"] = np.random.uniform(5, 15, size=xstats.index.size)
pyopia.statistics.add_best_guesses_to_stats(stats)
stats

In [ ]:
#
# Load image stats - contains whole-image statistics
#
image_stats = pyopia.io.load_image_stats(stats_file)
image_stats

In [ ]:
#
# Get particles with high probability of being copepods
#
xstats = xstats.where(xstats["probability_copepod"] > 0.7).dropna(dim="index")
xstats

In [ ]:
#
# Plot volume distribution from xstats - only copepods [uL / sample vol.]
#
dias, vd_total = pyopia.statistics.vd_from_stats(xstats, conf["general"]["pixel_size"])

plt.plot(
    dias,
    vd_total,
)
plt.xscale("log")
plt.xlabel("ECD [um]")
plt.ylabel("Volume Distribution [uL/sample vol.]")
plt.show()

In [ ]:
#
# Plot volume distribution from xstats - only copepods [uL / L]
# NB: This needs the correct number of images to get the total sample volume right - use image_stats for that
#
path_length_mm = 30
pixel_size = conf["general"]["pixel_size"]

# Get number of images
nimages = image_stats.timestamp.size

# Get raw image shape
imx, imy, _ = xstats.attrs["raw_image_shape"]

# Calculate sample volume
sample_volume = pyopia.statistics.get_sample_volume(
    pixel_size, path_length_mm, imx=imx, imy=imy
)

# Convert to uL / L
vd_total_scaled = vd_total / (sample_volume * nimages)

plt.plot(dias, vd_total_scaled, "k")
plt.xscale("log")
plt.xlabel("ECD [um")
plt.ylabel("Volume distribution [uL/L]")
plt.xlim(80, 12000)

In [ ]:
#
# Create montage of copepod particles
#
im_mont = pyopia.statistics.make_montage(
    xstats.to_pandas(),
    pixel_size=pixel_size,
    roidir=str(proj_dir / Path(conf["steps"]["statextract"]["export_outputpath"])),
    auto_scaler=500,
    msize=1024,
    maxlength=100000,
    crop_stats=None,
    eyecandy=True,
)

pyopia.plotting.montage_plot(im_mont, pixel_size)
plt.title(f"Montage based on copepod particles (N={xstats.index.size})")

In [ ]:
#
# Calculate total number of copepods per liter based on total volume sampled
#
total_sample_volume = sample_volume * nimages
num_copepods = xstats.index.size
print(f"Total number of images: {nimages}")
print(f"Total volume sampled: {total_sample_volume:.1f} (L)")
print(f"Number of copepods: {num_copepods}")
print(f"Copepods per liter: {num_copepods / total_sample_volume:.2g} (#/L)")

In [ ]:
#
# Calculate total number of copepods per liter based on total volume sampled
# with custom function defined at the top of this notebook
#
calculcate_copepod_concentration(
    xstats,
    image_stats,
    cop_prob_lim=0.7,
    path_length_mm=30,
)

In [ ]:
#
# Create EcoTaxa import bundle from potential Copepod particles
#
ecotaxa_export_bundle_filename = "ecotaxa_test_import.zip"
ecotaxa_export = pyopia.dataexport.ecotaxa.EcotaxaExporter()
ecotaxa_export.create_bundle(
    xstats, export_filename=ecotaxa_export_bundle_filename, make_label_folders=False,
    roi_dir=proj_dir / Path("roi")
)

In [ ]:
# There should be a new file ecotaxa_test_import.zip, containing particle image pngs 
# and particle statistics tsv file. 
with zipfile.ZipFile(ecotaxa_export_bundle_filename, 'r') as zip_ref:
    print("\n".join(zip_ref.namelist()))